In [1]:
from pyspark import SparkContext, SparkConf
sc = SparkContext(appName='project')
sc

<SparkContext master=local[*] appName=project>

In [2]:
rdd = sc.sequenceFile('datasets/part-00000')

In [4]:
def document_counter(item):
    words_brute = item[1].strip().split()
    words = [i.lower() for i in words_brute if (not any(j.isdigit() for j in i) and len(i) > 3 and (not("——") in i) and (not ("(") in i) and (not (")") in i) and (not ("www") in i))]
    
    if(scope == 0):
        if("hyundai" in words and "honda" in words):
            return [(i, 1) for i in set(words)]
    elif(scope == 1):
        if("hyundai" in words and "honda" not in words):
            return [(i, 1) for i in set(words)]
    else:
        if("hyundai" not in words and "honda" in words):
            return [(i, 1) for i in set(words)]
    
    return []


def word_counter(item):
    words_brute = item[1].strip().split()
    words = [i.lower() for i in words_brute if (not any(j.isdigit() for j in i) and len(i) > 3 and (not("——") in i) and (not ("(") in i) and (not (")") in i) and (not ("www") in i))]
    
    if(scope == 0):
        if("hyundai" in words and "honda" in words):
            return [(i, 1) for i in words]
    elif(scope == 1):
        if("hyundai" in words and "honda" not in words):
            return [(i, 1) for i in words]
    else:
        if("hyundai" not in words and "honda" in words):
            return [(i, 1) for i in words]
    return []


def count_words(result ,item):
    return result+item

# Variable scope -> 0 = Words together, 1 = Hyundai alone, 2 = Honda alone
scope = 0
docs_together = rdd.flatMap(document_counter).reduceByKey(count_words)
words_together = rdd.flatMap(word_counter).reduceByKey(count_words)

scope = 1
docs_hyundai = rdd.flatMap(document_counter).reduceByKey(count_words)
words_hyundai = rdd.flatMap(word_counter).reduceByKey(count_words)

scope = 2
docs_honda = rdd.flatMap(document_counter).reduceByKey(count_words)
words_honda = rdd.flatMap(word_counter).reduceByKey(count_words)


In [5]:
import math

def idf(item):
    return item[0], math.log(N/item[1],10)
    
def freq(item):
    return item[0], math.log(1 + item[1], 10)

def relevancy(item):
    return item[0], item[1][0] * item[1][1]

N = docs_honda.count()
rdd_idf_together = docs_together.map(idf)
N = docs_hyundai.count()
rdd_idf_hyundai = docs_hyundai.map(idf)
N = docs_together.count()
rdd_idf_honda = docs_honda.map(idf)

rdd_freq_together = words_together.map(freq)
rdd_freq_hyundai = words_hyundai.map(freq)
rdd_freq_honda = words_honda.map(freq)


rdd_relevancy_together = rdd_idf_together.join(rdd_freq_together).map(relevancy)
rdd_relevancy_hyundai = rdd_idf_hyundai.join(rdd_freq_hyundai).map(relevancy)
rdd_relevancy_honda = rdd_idf_honda.join(rdd_freq_honda).map(relevancy)

In [6]:
import pandas as pd
top_100_together = rdd_relevancy_together.takeOrdered(100, key=(lambda x: -x[1]))
result_together = pd.DataFrame(top_100_together, columns=(['word', 'relevancy']))
result_together.to_excel("tables/together.xlsx")
result_together.head(10)

,word,relevancy
0,mecanico,12.953494
1,serie,12.262597
2,requerente:,11.506755
3,trator,11.187739
4,carregadeira,10.875380
5,requerido:,10.727789
6,automatico,10.387358
7,alcool,10.348977
8,disse:,10.242844
9,carros-,10.232913


In [7]:
top_100_hyundai = rdd_relevancy_hyundai.takeOrdered(100, key=(lambda x: -x[1]))
result_hyundai = pd.DataFrame(top_100_hyundai, columns=(['word', 'relevancy']))
result_hyundai.to_excel("tables/hyundai.xlsx")
result_hyundai.head(10)

,word,relevancy
0,turbina,14.421680
1,remanufaturada,12.842796
2,diesel,12.830815
3,retifica,12.754393
4,disse:,12.706275
5,conversa,12.099730
6,responder,11.958560
7,oficina,10.970483
8,injetora,10.925580
9,revisão,10.686782


In [8]:
top_100_honda = rdd_relevancy_honda.takeOrdered(100, key=(lambda x: -x[1]))
result_honda = pd.DataFrame(top_100_honda, columns=(['word', 'relevancy']))
result_honda.to_excel("tables/honda.xlsx")
result_honda.head(10)

,word,relevancy
0,dating,12.589664
1,loans,12.430323
2,barrocas,11.751619
3,blokton,11.464962
4,payday,11.292125
5,title,10.895610
6,cams,10.647363
7,brides,10.638222
8,credit,10.358487
9,inicial:,10.310102


In [9]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

wordcloud = WordCloud(width=1500, height= 1500)
cloud_together = wordcloud.generate_from_frequencies(frequencies=dict(top_100_together))
cloud_together.to_file("wordclouds/together.png")

cloud_hyundai = wordcloud.generate_from_frequencies(frequencies=dict(top_100_hyundai))
cloud_hyundai.to_file("wordclouds/hyundai.png")

cloud_honda = wordcloud.generate_from_frequencies(frequencies=dict(top_100_honda))
cloud_honda.to_file("wordclouds/honda.png")
